In [21]:
import logging
from pathlib import Path

import numpy as np
import pandas as pd
from pydantic import BaseModel, Field, ValidationError
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [22]:

# ── Configuration ──────────────────────────────────────────────────────────────

class Config(BaseModel):
    play_csv: Path = Field(..., description="/media/picakse/Windows-SSD/Users/jpran/Downloads/googleplaystore.csv")
    ios_csv: Path = Field(..., description="/media/picakse/Windows-SSD/Users/jpran/Downloads/AppleStore.csv")
    random_state: int = 42
    test_size: float = 0.3
    n_jobs: int = -1

# ── Logging Setup ─────────────────────────────────────────────────────────────

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(levelname)s %(message)s")

# ── Data Loading & Validation ─────────────────────────────────────────────────

def load_data(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    logging.info(f"Loaded {len(df):,} rows from {path.name}")
    return df

# ── Preprocessing Pipeline ────────────────────────────────────────────────────


In [23]:

def build_preprocessor(numerical_cols, categorical_cols):
    num_pipeline = Pipeline([
        ("impute_mean", SimpleImputer(strategy="mean")),
        ("scale", StandardScaler()),
    ])
    cat_pipeline = Pipeline([
        ("impute_const", SimpleImputer(strategy="constant", fill_value="missing")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ])
    preprocessor = ColumnTransformer([
        ("num", num_pipeline, numerical_cols),
        ("cat", cat_pipeline, categorical_cols),
    ], remainder="drop")
    return preprocessor

# ── Modeling ──────────────────────────────────────────────────────────────────


In [24]:

def evaluate_models(X, y, preprocessor, config: Config):
    models = {
        "RandomForest": RandomForestClassifier(n_estimators=200, random_state=config.random_state, n_jobs=config.n_jobs),
        "GradientBoosting": GradientBoostingClassifier(random_state=config.random_state),
    }
    results = {}
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=config.random_state)
    for name, model in models.items():
        pipe = Pipeline([
            ("preproc", preprocessor),
            ("clf", model),
        ])
        scores = cross_val_score(pipe, X, y, cv=cv, scoring="f1_weighted", n_jobs=config.n_jobs)
        results[name] = scores
        logging.info(f"{name} F1-weighted CV: {scores.mean():.3f} ± {scores.std():.3f}")
    return results


In [25]:

# ── Main Workflow ─────────────────────────────────────────────────────────────

def main():
    try:
        config = Config(
            play_csv=Path("/media/picakse/Windows-SSD/Users/jpran/Downloads/googleplaystore.csv"),
            ios_csv=Path("/media/picakse/Windows-SSD/Users/jpran/Downloads/AppleStore.csv"),
        )
    except ValidationError as e:
        logging.error("Configuration error:\n%s", e)
        return

    # Load and merge datasets if needed; here illustrate Play store only
    df = load_data(config.play_csv)
    # Filter to English and free apps
    df = df[df["Type"] == "Free"].copy()
    df = df[df["App"].str.encode("ascii", errors="ignore").str.len() - df["App"].str.len().abs() <= 3]
    # Drop duplicates, missing target
    df.drop_duplicates(subset=["App"], inplace=True)
    df.dropna(subset=["Rating", "Reviews", "Category"], inplace=True)

    # Feature & target
    df["Reviews"] = df["Reviews"].astype(int)
    df["Installs"] = df["Installs"].str.replace(r"[+,]", "", regex=True).astype(int)
    df["Price"] = df["Price"].astype(float)
    X = df[["Rating", "Reviews", "Installs", "Price", "Category", "Content Rating"]]
    y = df["Genres"]  # or other binary target

    # Remove classes with fewer than 2 samples
    value_counts = y.value_counts()
    valid_classes = value_counts[value_counts >= 2].index
    mask = y.isin(valid_classes)
    X = X[mask]
    y = y[mask]

    numerical_cols = ["Rating", "Reviews", "Installs", "Price"]
    categorical_cols = ["Category", "Content Rating"]

    preprocessor = build_preprocessor(numerical_cols, categorical_cols)

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=config.test_size, stratify=y, random_state=config.random_state
    )

    # Evaluate models
    evaluate_models(X_train, y_train, preprocessor, config)

    # Final fit and report
    final_pipe = Pipeline([
        ("preproc", preprocessor),
        ("clf", RandomForestClassifier(n_estimators=200, random_state=config.random_state, n_jobs=config.n_jobs)),
    ])
    final_pipe.fit(X_train, y_train)
    y_pred = final_pipe.predict(X_test)
    logging.info("Final Confusion Matrix:\n%s", confusion_matrix(y_test, y_pred))
    logging.info("Final Classification Report:\n%s", classification_report(y_test, y_pred))

if __name__ == "__main__":
    main()


2025-08-02 18:51:59,850 INFO Loaded 10,841 rows from googleplaystore.csv
/home/picakse/my_tensorflow_project/tf_gpu/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
2025-08-02 18:52:02,279 INFO RandomForest F1-weighted CV: 0.769 ± 0.006
/home/picakse/my_tensorflow_project/tf_gpu/lib/python3.12/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
2025-08-02 18:52:44,303 INFO GradientBoosting F1-weighted CV: 0.762 ± 0.005
2025-08-02 18:52:44,785 INFO Final Confusion Matrix:
[[46  0  6 ...  0  0  1]
 [ 0  0  0 ...  0  0  0]
 [10  0  2 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0 20  0]
 [ 3  0  0 ...  0  0  2]]
/home/picakse/my_tensorflow_project/tf_gpu/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMe

In [28]:
import joblib

# If you have already run main(), but final_pipe is not available,
# you need to define or extract it here. 
# For demonstration, you can re-create and fit final_pipe as in main():

# Re-create config and preprocessor as in main()
config = Config(
	play_csv=Path("/media/picakse/Windows-SSD/Users/jpran/Downloads/googleplaystore.csv"),
	ios_csv=Path("/media/picakse/Windows-SSD/Users/jpran/Downloads/AppleStore.csv"),
)
df = load_data(config.play_csv)
df = df[df["Type"] == "Free"].copy()
df = df[df["App"].str.encode("ascii", errors="ignore").str.len() - df["App"].str.len().abs() <= 3]
df.drop_duplicates(subset=["App"], inplace=True)
df.dropna(subset=["Rating", "Reviews", "Category"], inplace=True)
df["Reviews"] = df["Reviews"].astype(int)
df["Installs"] = df["Installs"].str.replace(r"[+,]", "", regex=True).astype(int)
df["Price"] = df["Price"].astype(float)
X = df[["Rating", "Reviews", "Installs", "Price", "Category", "Content Rating"]]
y = df["Genres"]
value_counts = y.value_counts()
valid_classes = value_counts[value_counts >= 2].index
mask = y.isin(valid_classes)
X = X[mask]
y = y[mask]
numerical_cols = ["Rating", "Reviews", "Installs", "Price"]
categorical_cols = ["Category", "Content Rating"]
preprocessor = build_preprocessor(numerical_cols, categorical_cols)
X_train, X_test, y_train, y_test = train_test_split(
	X, y, test_size=config.test_size, stratify=y, random_state=config.random_state
)
final_pipe = Pipeline([
	("preproc", preprocessor),
	("clf", RandomForestClassifier(n_estimators=200, random_state=config.random_state, n_jobs=config.n_jobs)),
])
final_pipe.fit(X_train, y_train)

# Save the trained pipeline
joblib.dump(final_pipe, 'model.joblib')
print("Saved trained model pipeline to model.joblib")


2025-08-02 18:53:53,866 INFO Loaded 10,841 rows from googleplaystore.csv


Saved trained model pipeline to model.joblib


In [30]:
import joblib

joblib.dump(final_pipe, 'model.joblib')
print("Saved trained model pipeline to model.joblib")


Saved trained model pipeline to model.joblib
